Using `SmugPyter` Batch Scripts
===============

![](jupysmug.png)

### Maintaining Local SmugMirror Directories

`SmugPyter` maintains local directories of SmugMug metadata. The directories
mirror your online folders and albums and contain:

1. Metadata in simple CSV TAB delimited text files.

2. Sample images.

This information is used to set:

1. Print size keywords.

2. Geotag keywords.

3. Dominant color keywords.

The SmugMirror is primarily an input to other programs including version control systems like `Git` and `Fossil` that
can track changes in text files.

Keeping local directories current is simple matter of running a few batch scripts.

### Clearing Changes Files

Keyword changes are staged in changes files. Changes files follow this naming convention.


Before updating local files it's a good idea to empty existing changes files.

In [1]:
import sys

# add smugpyter scripts to path
sys.path.append('C:\\mp\\jupyter\\smugpyter\\notebooks')
#print(sys.path)

import smugpyter 

In [2]:
# Empty all CSV changes files: assumes (smugpyter) location is on sys.path.
smug = smugpyter.SmugPyter()
print(smug.reset_all_changes_files(smug.local_directory))

(81, 81)


Reseting changes files is also available via the batch script.

     ResetAllChangesFiles.bat

### Refreshing Album Metadata

The leaf directories in SmugMirror contain CSV TAB delimited metadata text files that contain
information about albums, images and dates. The files are named like:

    ainfo-<deblanked-album-name>-<smugmug-album-key>-<key-case-mask>.txt
    manifest-<deblanked-album-name>-<smugmug-album-key>-<key-case-mask>.txt
    realdate-<deblanked-album-name>-<smugmug-album-key>-<key-case-mask>.txt
    
Here are some examples:
    
    ainfo-BeenThereDoneThat-w5PKZp-e.txt
    manifest-BeenThereDoneThat-w5PKZp-e.txt
    realdate-BeenThereDoneThat-w5PKZp-e.txt

These files are key inputs to functions in `SmugPyter` and its subclasses. Here is how you keep them updated.

In [ ]:
# Refresh album TAB delimited files: assumes (smugpyter) location is on sys.path.
import smugpyter

smug = smugpyter.SmugPyter()
smug.download_album_metadata()

Refreshing metadata is also available via the batch script.

     RefreshMetadata.bat

### Refreshing Recently Changed Metadata

Album metadata is fairly stable. Usually you only want to collect recently changed albums.
The following gathers recent changes. The default is to look fourteen days back. The default is
set in the config file.

In [ ]:
# Refresh album TAB delimited files of recently changed online galleries.
import smugpyter

smug = smugpyter.SmugPyter()

# smug.days_before set in config 
smug.download_album_metadata(smug.days_before)

Refreshing recently changed metadata is also available via the batch script.

     RefreshRecentMetadata.bat

### Collecting Sample Images from SmugMug

Before image color and geottag keys can be computed it's necessary to gather sample images
from SmugMug. Sample images are small `jpeg` versions of larger online images. When sample images
are downloaded they are renamed as:

     <smugmug-image-key>-<key-case-mask>-<file-name-with-blanks-replaced-with-hyphens>.jpg
     
Here are some examples.

     vF4Bwpg-14-ghanasco-track-team-1977.jpg
     ZF5SPhL-31-me-near-three-island-crossing.jpg
     PSdGFkr-30-aub-porthole-sea-tower.jpg

The SmugMug image key is unique across all images for a given user account. The SmugMug image
key is case sensitive hence a case mask is attached to preserve the case on systems like
Windows that often ignore file name case.

In [ ]:
# Download new sample images: assumes (smugpyter) is on sys.path.
# Run after refreshing metadata files.
import smugpyter

smug = smugpyter.SmugPyter()
#smug.yammer = True
print(smug.update_all_sample_images(smug.local_directory))

Sample images can also be collected with the batch file.

     UpdateSampleImages.bat

### Computing Geotag, Print Size and Dominant Color Keys

Getotag, print size and dominant color keys are computed with:

In [3]:
# Update CSV TAB delimited geotag, print size and color keyword 
# changes files. Run after refreshing metadata files and updating
# sample images. Assumes (smugpyter, geotagkeys, printkeys, colorkeys)
# are on sys.path.

import geotagkeys
import printkeys
import colorkeys

# subclasses of SmugPyter
gk = geotagkeys.GeotagKeys(log_start=True)
pk = printkeys.PrintKeys()
ck = colorkeys.ColorKeys(verbose=True)

# local directory in config - set in superclass
root = pk.local_directory

# reset all changes files
print(gk.reset_all_changes_files(root))

# update changes files - order matters
print(gk.update_all_geotag_keyword_changes(root))

pk.merge_changes = True
print(pk.update_all_size_keyword_changes(root))

SmugPyter started: Mon Feb 20 10:56:35 2023
(81, 81)
processing geotag keys
unable to reverse geocode -34.217588888889,169.25476111111
unable to reverse geocode -34.216702777778,169.21832222222
(6775, 1)
processing print size leys
(6775, 210)


Color keys is failing (**Feb 20, 2023**) - I recently upgraded my Anaconada system
and as usual some older scripts stopped working. The source of the problem was a change
of case in the name of a `webcolors` function - jesus people - you don't rename common utilities
without a really good reason!

In [4]:
ck.merge_changes = True
print(ck.update_all_color_keyword_changes(root))

processing color keys
me omar sharif kanab.jpg | 0_darkslategray; 4x5; geotagged; kanab; me; omar; sharif; usa; utah
(6775, 1)


Finally, geotag, print size and color keywords can be set with:

     UpdateSmugKeywords.bat

### API PATCH PROBLEMS: Posting Key Change to SmugMug

Prior calculations write key changes to change files in each sample image
directory and to text files in the root. The root files are:

     GeoChanges.txt
     SizeChanges.txt
     AllChanges.txt
     
`GeoChanges.txt` and `SizeChanges.txt` are loaded before `AllChanges.txt`.  All key changes
posted to SmugMug come from `AllChanges.txt`

**NOTE: SMUGMUG PATCH API REQUESTS ARE BROKEN AGAIN!** I have
been posting key changes by cutting and pasting `smugpyter` outputs to
SmugMug by hand. Hence, the changes to limit and centralize all changes
to one spot.

**TODO: Find out why `PATCH` does not work!**

After setting local changes files the keywords can be pushed to SmugMug with:

In [ ]:
# Run after refreshing metadata files and updating keyword changes files: 
# assumes (smugpyter) is on sys.path
import smugpyter

#smug = smugpyter.SmugPyter()
#print(smug.update_all_keyword_changes(smug.local_directory))

Pushing keyword changes to SmugMug can be done with:

    ChangeKeywords.bat

## Next on the Agenda!

The local directories maintained by `SmugPyter` are mainly inputs to other programs. I am already
using these directories to build an SQLite database with J. See the group `MirrorXref` in 
the [JOD](https://analyzethedatanotthedrivel.org/the-jod-page/) dictionary [smugpyter](https://github.com/bakerjd99/joddumps) for details.

Remember, always [Analyze the Data not the Drivel](https://analyzethedatanotthedrivel.org/).

John Baker, Meridian Idaho